In [ ]:
import pandas as pd

In [ ]:
df = pd.read_excel("/Users/issackondreddy/Desktop/Education/Projects/Cancer Detection/Docs/MA Results.xlsx")
for function in df['Benchmark Function'].unique():
    func_df = df[df['Benchmark Function'] == function]
    for col in ['Best Fitness', 'Average Fitness', 'Standard Deviation']:
        min_val = func_df[col].min()
        max_val = func_df[col].max()
        df.loc[func_df.index, col + ' Norm'] = (func_df[col] - min_val) / (max_val - min_val)
    # Invert 'Convergence Generation'
    min_val = func_df['Convergence Generation'].min()
    max_val = func_df['Convergence Generation'].max()
    df.loc[func_df.index, 'Convergence Generation Norm'] = 1 - (func_df['Convergence Generation'] - min_val) / (max_val - min_val)

# Rank the normalized values
for col in ['Best Fitness Norm', 'Average Fitness Norm', 'Standard Deviation Norm', 'Convergence Generation Norm']:
    df[col + ' Rank'] = df.groupby('Benchmark Function')[col].rank()

# Calculate weighted score with your weights
df['Weighted Score'] = (
    df['Best Fitness Norm Rank'] * 0.5 + 
    df['Average Fitness Norm Rank'] * 0.3 + 
    df['Convergence Generation Norm Rank'] * 0.1 +
    df['Standard Deviation Norm Rank'] * 0.1
)

# Sort and select top MAs
top_mAs = df.sort_values(by='Weighted Score').groupby('Benchmark Function').head(5)


In [ ]:
df.head()

In [ ]:
output_file_path = "/Users/issackondreddy/Desktop/Education/Projects/Cancer Detection/Docs/MA_Results_Summary.xlsx"

# Save the DataFrame to a CSV file
df.to_excel(output_file_path, index=False)

In [8]:
clean_df = df.dropna(subset=['Algorithm', 'Weighted Score']).copy()

# Calculate the min and max for normalization
min_score = clean_df['Weighted Score'].min()
max_score = clean_df['Weighted Score'].max()

# Use .loc to avoid SettingWithCopyWarning
clean_df.loc[:, 'Normalized Weighted Score'] = (clean_df['Weighted Score'] - min_score) / (max_score - min_score)

# Sort by 'Normalized Weighted Score' in ascending order to get the best scores first
sorted_clean_df = clean_df.sort_values(by='Normalized Weighted Score', ascending=True)

# Drop duplicate 'Algorithm' entries, keeping the first occurrence
top_unique_MAs = sorted_clean_df.drop_duplicates(subset='Algorithm')

# Select the top 3 MAs with the lowest normalized weighted score
top_3_MAs = top_unique_MAs.head(3)

# Display the top 3 MAs
print(top_3_MAs[['Algorithm', 'Benchmark Function', 'Normalized Weighted Score']])

                 Algorithm Benchmark Function  Normalized Weighted Score
0        Genetic Algorithm             Sphere                   0.000000
64    Salp swarm Algorithm             Sphere                   0.154321
20  Differential Evolution             Sphere                   0.225309
